# Capstone Project - Car Accident Severity Prediction

This is the notebook for car accident severity prediction project :-)

In [64]:
import pandas as pd
import numpy as np

In [65]:
print("Hello Capstone Project Course!")

Hello Capstone Project Course!


In [66]:
df_before = pd.read_csv('Data-Collisions.csv')

/Users/mars/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (33) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [67]:
df_before.head()

,SEVERITYCODE,X,Y,OBJECTID,INCKEY,COLDETKEY,REPORTNO,STATUS,ADDRTYPE,INTKEY,...,ROADCOND,LIGHTCOND,PEDROWNOTGRNT,SDOTCOLNUM,SPEEDING,ST_COLCODE,ST_COLDESC,SEGLANEKEY,CROSSWALKKEY,HITPARKEDCAR
0,2,-122.323148,47.703140,1,1307,1307,3502005,Matched,Intersection,37475.0,...,Wet,Daylight,NaN,NaN,NaN,10,Entering at angle,0,0,N
1,1,-122.347294,47.647172,2,52200,52200,2607959,Matched,Block,NaN,...,Wet,Dark - Street Lights On,NaN,6354039.0,NaN,11,From same direction - both going straight - bo...,0,0,N
2,1,-122.334540,47.607871,3,26700,26700,1482393,Matched,Block,NaN,...,Dry,Daylight,NaN,4323031.0,NaN,32,One parked--one moving,0,0,N
3,1,-122.334803,47.604803,4,1144,1144,3503937,Matched,Block,NaN,...,Dry,Daylight,NaN,NaN,NaN,23,From same direction - all others,0,0,N
4,2,-122.306426,47.545739,5,17700,17700,1807429,Matched,Intersection,34387.0,...,Wet,Daylight,NaN,4028032.0,NaN,10,Entering at angle,0,0,N


In [68]:
df_before.isnull().sum()

SEVERITYCODE           0
X                   5334
Y                   5334
OBJECTID               0
INCKEY                 0
COLDETKEY              0
REPORTNO               0
STATUS                 0
ADDRTYPE            1926
INTKEY            129603
LOCATION            2677
EXCEPTRSNCODE     109862
EXCEPTRSNDESC     189035
SEVERITYCODE.1         0
SEVERITYDESC           0
COLLISIONTYPE       4904
PERSONCOUNT            0
PEDCOUNT               0
PEDCYLCOUNT            0
VEHCOUNT               0
INCDATE                0
INCDTTM                0
JUNCTIONTYPE        6329
SDOT_COLCODE           0
SDOT_COLDESC           0
INATTENTIONIND    164868
UNDERINFL           4884
WEATHER             5081
ROADCOND            5012
LIGHTCOND           5170
PEDROWNOTGRNT     190006
SDOTCOLNUM         79737
SPEEDING          185340
ST_COLCODE            18
ST_COLDESC          4904
SEGLANEKEY             0
CROSSWALKKEY           0
HITPARKEDCAR           0
dtype: int64

## 📖 Introduction/Business Problem ##

 With an increasing number of car accidents happened in our community, a model is required to be developed in order to predict how severe an accident would be based on weather condition, road condition, location, etc. This project is focusing on developing a machine learning model to predict the severity of a car collision based on big dataset gathered from the community. 

## 🔢 Data ##

### *Severity codes are as follows:*###

*0: Little to no Probability (Clear Conditions)*

*1: Very Low Probability — Chance or Property Damage*

*2: Low Probability — Chance of Injury*

*3: Mild Probability — Chance of Serious Injury*

*4: High Probability — Chance of Fatality*

### ❎ The original dataset has the following problems: ###

-- It has too many attributes (37)

-- It is an unbalanced dataset (SEVERITYCODE column)

### 🔧 So some changes are applied to this dataset: ###

-- The **independent variables** are changed to WEATHER, ROADCOND, LIGHTCOND

-- Balance the **dependent variables** SEVERITYCODE which now 1 and 2 categories have equal amount of data

## 🔮 Data Conversion ##

 ### **❎ Deal with unbalanced SEVERITYCODE**###

In [69]:
df_before['SEVERITYCODE'].value_counts()

1    136485
2     58188
Name: SEVERITYCODE, dtype: int64

As we can see the SEVERITYCODE column is unbalanced and category '1' is nearly 2 more times than the category '2'

In [70]:
from sklearn.utils import resample

df_before_maj = df_before[df_before.SEVERITYCODE==1]
df_before_min = df_before[df_before.SEVERITYCODE==2]

df_before_maj_upsampled = resample(df_before_maj, 
                                 replace=True,     # sample with replacement
                                 n_samples=58188,    # to match majority class
                                 random_state=123) # reproducible results
 
# Combine majority class with upsampled minority class
balanced_df = pd.concat([df_before_maj_upsampled, df_before_min])
balanced_df.SEVERITYCODE.value_counts()

2    58188
1    58188
Name: SEVERITYCODE, dtype: int64

#### 🌟 Now the dataset is balanced! ####

### **❎ Deal with too many attributes**###

In [71]:
df = pd.DataFrame(df_before[['SEVERITYCODE','WEATHER','ROADCOND','LIGHTCOND']])
df.head()

,SEVERITYCODE,WEATHER,ROADCOND,LIGHTCOND
0,2,Overcast,Wet,Daylight
1,1,Raining,Wet,Dark - Street Lights On
2,1,Overcast,Dry,Daylight
3,1,Clear,Dry,Daylight
4,2,Raining,Wet,Daylight


In [72]:
df_before['WEATHER'].value_counts()

Clear                       111135
Raining                      33145
Overcast                     27714
Unknown                      15091
Snowing                        907
Other                          832
Fog/Smog/Smoke                 569
Sleet/Hail/Freezing Rain       113
Blowing Sand/Dirt               56
Severe Crosswind                25
Partly Cloudy                    5
Name: WEATHER, dtype: int64

In [73]:
df_before['ROADCOND'].value_counts()

Dry               124510
Wet                47474
Unknown            15078
Ice                 1209
Snow/Slush          1004
Other                132
Standing Water       115
Sand/Mud/Dirt         75
Oil                   64
Name: ROADCOND, dtype: int64

In [74]:
df_before['LIGHTCOND'].value_counts()

Daylight                    116137
Dark - Street Lights On      48507
Unknown                      13473
Dusk                          5902
Dawn                          2502
Dark - No Street Lights       1537
Dark - Street Lights Off      1199
Other                          235
Dark - Unknown Lighting         11
Name: LIGHTCOND, dtype: int64

In [75]:
encoding_weather = {'WEATHER':{'Clear': 1, 'Partly Cloudy': 2, 'Overcast':3,  'Fog/Smog/Smoke':4, 'Severe Crosswind':5, 'Raining':6, 'Sleet/Hail/Freezing Rain':7, 'Blowing Sand/Dirt':8, 'Snowing':9,'Other':10,'Unknown':11}}
df.replace(encoding_weather, inplace=True)
df.head()

,SEVERITYCODE,WEATHER,ROADCOND,LIGHTCOND
0,2,3.0,Wet,Daylight
1,1,6.0,Wet,Dark - Street Lights On
2,1,3.0,Dry,Daylight
3,1,1.0,Dry,Daylight
4,2,6.0,Wet,Daylight


In [76]:
encoding_roadcond = {'ROADCOND':{'Dry': 1, 'Sand/Mud/Dirt':2, 'Oil':3, 'Wet':4,'Standing Water':5,'Snow/Slush':6,'Ice':7,'Other':8,'Unknown':9}}
df.replace(encoding_roadcond, inplace=True)
df.head()

,SEVERITYCODE,WEATHER,ROADCOND,LIGHTCOND
0,2,3.0,4.0,Daylight
1,1,6.0,4.0,Dark - Street Lights On
2,1,3.0,1.0,Daylight
3,1,1.0,1.0,Daylight
4,2,6.0,4.0,Daylight


In [77]:
encoding_lightcond = {'LIGHTCOND':{'Daylight':1, 'Dawn':2, 'Dusk':3, 'Dark - Street Lights On':4, 'Dark - Street Lights Off':5, 'Dark - No Street Lights':6, 'Dark - Unknown Lighting':7, 'Other':8,'Unknown':9 }}
df.replace(encoding_lightcond, inplace=True)
df.head()

,SEVERITYCODE,WEATHER,ROADCOND,LIGHTCOND
0,2,3.0,4.0,1.0
1,1,6.0,4.0,4.0
2,1,3.0,1.0,1.0
3,1,1.0,1.0,1.0
4,2,6.0,4.0,1.0
